In [1]:
import pandas as pd
import os

In [2]:
strategy_path = './backtesting_pipeline/strategies/BbandsCross'

wfo_dir = os.path.join(strategy_path, 'wfo')
pa_dir = os.path.join(strategy_path, 'preliminar_analysis')
random_test_dir = os.path.join(strategy_path, 'random_test')
montecarlo_dir = os.path.join(strategy_path, 'montecarlo')

pa_filter_performance = pd.read_csv(
    os.path.join(pa_dir, 'filter_performance.csv')
)

pa_filter_performance['strategy'] = pa_filter_performance['ticker'] + '_' + pa_filter_performance['interval'].astype(str)
del pa_filter_performance['ticker']
del pa_filter_performance['interval']


wfo_filter_performance = pd.read_csv(
    os.path.join(wfo_dir, 'filter_performance.csv')
)

wfo_filter_performance['strategy'] = wfo_filter_performance['ticker'] + '_' + wfo_filter_performance['interval'].astype(str)

del wfo_filter_performance['ticker']
del wfo_filter_performance['interval']

In [3]:
import re

def compare_trades(dir_1, dir_2, name_to_col_1, name_to_col_2):

    all_trades = {}
    equity = None
    all_trades_to_compare = {}
    equity_to_compare = None

    pattern = r"^[A-Z0-9]+\.[a-z]+_[0-9]+$|^[A-Z]+_[0-9]+$"
    
    files_1 = os.listdir(dir_1)
    files_1 = [d for d in files_1 if re.match(pattern, d)]
    
    files_2 = os.listdir(dir_2)
    files_2 = [d for d in files_2 if re.match(pattern, d)]
    
    files = [value for value in files_1 if value in files_2]

    for dir in files:
        
        trades = pd.read_csv(
            os.path.join(dir_1, dir, 'trades.csv')
        )
        equity = pd.read_csv(
            os.path.join(dir_1, dir, 'equity.csv'), index_col=0
        )
        
        trades = pd.merge(
            trades,
            equity,
            left_on='ExitTime',
            right_index=True,
            how='inner'
        )
        trades['ReturnPct'] = trades['PnL'] / trades['Equity'].shift(1)
        
        all_trades[dir] = trades
        
        trades_to_compare = pd.read_csv(
            os.path.join(dir_2, dir,  'trades.csv')
        )
        
        equity_to_compare = pd.read_csv(
            os.path.join(dir_2, dir, 'equity.csv') , index_col=0
        )
        trades_to_compare = pd.merge(
            trades_to_compare,
            equity_to_compare,
            left_on='ExitTime',
            right_index=True,
            how='inner'
        )
        trades_to_compare['ReturnPct'] = trades_to_compare['PnL'] / trades_to_compare['Equity'].shift(1)
        
        all_trades_to_compare[dir] = trades_to_compare
        
    performance = pd.DataFrame()

    for k in all_trades.keys():
        df_trades = all_trades[k]
        trades_profit_percent = df_trades.ReturnPct.sum() * 100
        long_trades_profit_percent = df_trades[df_trades['Size']>0].ReturnPct.sum() * 100
        short_trades_profit_percent = df_trades[df_trades['Size']<0].ReturnPct.sum() * 100

        df_trades_to_compare = all_trades_to_compare[k]
        trades_to_compare_profit_percent = df_trades_to_compare.ReturnPct.sum() * 100
        long_trades_to_compare_profit_percent = df_trades_to_compare[df_trades_to_compare['Size']>0].ReturnPct.sum() * 100
        short_trades_to_compare_profit_percent = df_trades_to_compare[df_trades_to_compare['Size']<0].ReturnPct.sum() * 100
        
        general_performance = (trades_profit_percent - trades_to_compare_profit_percent) 
        long_performance = (long_trades_profit_percent - long_trades_to_compare_profit_percent)
        short_performance = (short_trades_profit_percent - short_trades_to_compare_profit_percent)
        
        random_test_df = pd.DataFrame({
            'strategy': [k],
            f'{name_to_col_1}_trades_profit_percent': [trades_profit_percent],
            f'{name_to_col_2}_trades_profit_percent': [trades_to_compare_profit_percent],
            f'{name_to_col_1}_general_performance_over_{name_to_col_2}%': [general_performance],
            
            f'{name_to_col_1}_long_trades_profit_percent': [long_trades_profit_percent],
            f'{name_to_col_2}_long_trades_profit_percent': [long_trades_to_compare_profit_percent],
            f'{name_to_col_1}_long_performance_over_{name_to_col_2}%': [long_performance],
            
            f'{name_to_col_1}_short_trades_profit_percent': [short_trades_profit_percent],
            f'{name_to_col_2}_short_trades_profit_percent': [short_trades_to_compare_profit_percent],
            f'{name_to_col_1}_short_performance_over_{name_to_col_2}%': [short_performance],
            
        })
        
        performance = pd.concat([performance, random_test_df])
        
    performance = performance.T
    performance.columns = performance.iloc[0]
    performance = performance[1:]
    return performance

# Pa VS WFO

In [4]:
compare_trades(
    dir_1=pa_dir,
    dir_2=wfo_dir,
    name_to_col_1='pa',
    name_to_col_2='wfo',
)

strategy,GBPUSD_16385,US100.cash_16387,US30.cash_16387,US500.cash_16387,USDCHF_16386
pa_trades_profit_percent,25.185918,39.936473,51.681466,14.003602,10.985325
wfo_trades_profit_percent,13.971933,24.655484,43.476826,11.631176,4.457409
pa_general_performance_over_wfo%,11.213985,15.280989,8.20464,2.372425,6.527915
pa_long_trades_profit_percent,13.384774,19.771128,17.892281,1.531699,7.178032
wfo_long_trades_profit_percent,6.984682,4.572139,7.302789,1.027699,2.235245
pa_long_performance_over_wfo%,6.400091,15.198989,10.589493,0.504,4.942787
pa_short_trades_profit_percent,11.801144,20.165346,33.789185,12.471903,3.807293
wfo_short_trades_profit_percent,6.987251,20.083346,36.174038,10.603478,2.222165
pa_short_performance_over_wfo%,4.813894,0.082,-2.384853,1.868425,1.585128


# Best VS Random

In [5]:
compare_trades(
    dir_1=pa_dir,
    dir_2=random_test_dir,
    name_to_col_1='pa',
    name_to_col_2='rt',
)

strategy,BTCUSD_16388,GBPUSD_16385,META_16387,NFLX_16386,T_16388,US100.cash_16387,US2000.cash_16385,US30.cash_16387,US500.cash_16387,USDCHF_16386
pa_trades_profit_percent,88.890934,25.185918,4.152913,6.355025,1.185272,39.936473,11.421526,51.681466,14.003602,10.985325
rt_trades_profit_percent,21.025631,1.819423,-3.453082,-3.890969,3.652517,7.126715,-17.312103,4.572312,0.322043,-1.700055
pa_general_performance_over_rt%,67.865303,23.366495,7.605994,10.245994,-2.467244,32.809758,28.73363,47.109153,13.681559,12.68538
pa_long_trades_profit_percent,34.733512,13.384774,1.163627,5.721225,1.39625,19.771128,5.049747,17.892281,1.531699,7.178032
rt_long_trades_profit_percent,10.144645,3.645379,1.575298,-2.777957,-1.511407,7.695221,-12.136364,3.807249,-0.456477,-1.106882
pa_long_performance_over_rt%,24.588867,9.739395,-0.411671,8.499182,2.907657,12.075907,17.186111,14.085032,1.988176,8.284914
pa_short_trades_profit_percent,54.157422,11.801144,2.989286,0.6338,-0.210977,20.165346,6.371779,33.789185,12.471903,3.807293
rt_short_trades_profit_percent,10.880986,-1.825956,-5.028379,-1.113011,5.163924,-0.568506,-5.175739,0.765063,0.778519,-0.593173
pa_short_performance_over_rt%,43.276436,13.627101,8.017665,1.746812,-5.374901,20.733851,11.547518,33.024121,11.693383,4.400466


# Montecarlo

In [6]:
all_dd = pd.DataFrame()
all_returns = pd.DataFrame()

for file in os.listdir(montecarlo_dir):

    mc = pd.read_csv(
        os.path.join(montecarlo_dir, file)
    )

    key = file.split('.')[0]

    mc = mc.T
    mc.columns = mc.iloc[0]
    mc = mc[1:]

    dd_df = pd.DataFrame(mc.loc['Drawdown (%)']).T
    return_df = pd.DataFrame(mc.loc['Final Return (%)']).T
    
    for column in dd_df.columns:
        dd_df = dd_df.rename(columns={column: f'dd_{column}'})

    for column in return_df.columns:
        return_df = return_df.rename(columns={column: f'ret_{column}'})
        
    return_df['strategy'] = key
    dd_df['strategy'] = key
        
        
    all_dd = pd.concat([all_dd, dd_df])
    all_returns = pd.concat([all_returns, return_df])

In [7]:
pa_filter_performance['method'] = 'pa'
wfo_filter_performance['method'] = 'wfo'

report = pd.concat(
    [pa_filter_performance, wfo_filter_performance,]
).sort_values(by='strategy')

report

,strategy,stability_ratio,trades,return,drawdown,return/dd,custom_metric,win_rate,avg_trade_percent,Duration,method
1,BTCUSD_16388,0.592293,59,194.804700,33.614615,5.795238,23.042220,69.491525,0.730036,1400 days 00:00:00,pa
0,GBPUSD_16385,0.961657,203,28.232457,4.643921,6.079444,26.602711,73.399015,0.062061,1397 days 00:00:00,pa
0,GBPUSD_16385,0.674648,174,13.371830,5.106096,2.618797,11.310442,69.540230,0.013849,1304 days 05:00:00,wfo
9,META_16387,0.857733,18,4.668025,2.028611,2.301094,4.538289,83.333333,1.189031,1396 days 06:00:00,pa
7,NFLX_16386,0.944625,26,7.148548,1.377684,5.188816,9.908991,73.076923,1.594544,1396 days 06:00:00,pa
8,T_16388,0.735815,11,5.534061,1.450615,3.814977,5.611501,72.727273,1.862671,1396 days 04:00:00,pa
3,US100.cash_16387,0.923890,70,45.308570,11.431794,3.963382,15.535644,77.142857,0.470011,1397 days 00:00:00,pa
1,US100.cash_16387,0.736553,49,19.062701,12.500060,1.525009,5.523955,71.428571,0.309093,1100 days 00:00:00,wfo
4,US2000.cash_16385,0.903556,173,11.851284,3.920573,3.022845,12.425672,68.786127,0.040119,1396 days 22:00:00,pa
2,US30.cash_16387,0.855561,62,65.899229,13.157729,5.008404,19.284829,74.193548,0.325862,1397 days 00:00:00,pa


In [8]:

report.sort_values(by='custom_metric', ascending=False).drop_duplicates(subset=['strategy'])

,strategy,stability_ratio,trades,return,drawdown,return/dd,custom_metric,win_rate,avg_trade_percent,Duration,method
0,GBPUSD_16385,0.961657,203,28.232457,4.643921,6.079444,26.602711,73.399015,0.062061,1397 days 00:00:00,pa
1,BTCUSD_16388,0.592293,59,194.804700,33.614615,5.795238,23.042220,69.491525,0.730036,1400 days 00:00:00,pa
2,US30.cash_16387,0.855561,62,65.899229,13.157729,5.008404,19.284829,74.193548,0.325862,1397 days 00:00:00,pa
3,US100.cash_16387,0.923890,70,45.308570,11.431794,3.963382,15.535644,77.142857,0.470011,1397 days 00:00:00,pa
4,US2000.cash_16385,0.903556,173,11.851284,3.920573,3.022845,12.425672,68.786127,0.040119,1396 days 22:00:00,pa
5,USDCHF_16386,0.824574,99,10.548979,2.989870,3.528240,12.175797,74.747475,0.096986,1397 days 00:00:00,pa
6,US500.cash_16387,0.918948,65,15.124762,4.479056,3.376775,11.565411,72.307692,0.474750,1397 days 00:00:00,pa
7,NFLX_16386,0.944625,26,7.148548,1.377684,5.188816,9.908991,73.076923,1.594544,1396 days 06:00:00,pa
8,T_16388,0.735815,11,5.534061,1.450615,3.814977,5.611501,72.727273,1.862671,1396 days 04:00:00,pa
9,META_16387,0.857733,18,4.668025,2.028611,2.301094,4.538289,83.333333,1.189031,1396 days 06:00:00,pa


In [9]:
report = pd.DataFrame()

report = pd.merge(
    pa_filter_performance, 
    all_returns,
    on='strategy',
)

report = pd.merge(
    report, 
    all_dd,
    on='strategy',
).T

report.columns = report.iloc[0]
report = report[1:] 
report

strategy,GBPUSD_16385,BTCUSD_16388,USDCHF_16386,NFLX_16386,T_16388,META_16387
stability_ratio,0.961657,0.592293,0.824574,0.944625,0.735815,0.857733
trades,203,59,99,26,11,18
return,28.232457,194.8047,10.548979,7.148548,5.534061,4.668025
drawdown,4.643921,33.614615,2.98987,1.377684,1.450615,2.028611
return/dd,6.079444,5.795238,3.52824,5.188816,3.814977,2.301094
custom_metric,26.602711,23.04222,12.175797,9.908991,5.611501,4.538289
win_rate,73.399015,69.491525,74.747475,73.076923,72.727273,83.333333
avg_trade_percent,0.062061,0.730036,0.096986,1.594544,1.862671,1.189031
Duration,1397 days 00:00:00,1400 days 00:00:00,1397 days 00:00:00,1396 days 06:00:00,1396 days 04:00:00,1396 days 06:00:00
method,pa,pa,pa,pa,pa,pa


In [ ]:
report.columns[0]